In [10]:
import math
import random
import numpy as np
import os
import fnmatch
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import shapelets as sha
import importlib
importlib.reload(sha)
from operator import itemgetter
lcs=[]
classes=[]
xs=np.arange(0,30)
def noisy(average_value):
    return average_value+(math.cos(random.randint(0,360)*(math.pi/180))*0.01*average_value)
no_per_class=10
for i in range(no_per_class):
    ys=[]
    peak=np.random.choice(xs[1:-1])
    for x in xs:
        y=noisy(2)
        ys.append(y)
    ys[peak]=ys[peak-1]=ys[peak+1]=noisy(5)
    lcs.append(np.stack((xs,ys)))
    classes.append("alpha")
for i in range(no_per_class):
    ys=[]
    peak=np.random.choice(xs[:-1])
    for x in xs:
        y=noisy(2)
        ys.append(y)
    ys[peak]=noisy(10)
    lcs.append(np.stack((xs,ys)))
    classes.append("beta")
for i in range(no_per_class):
    ys=[]
    peak=np.random.choice(xs[:-3])
    for x in xs:
        y=noisy(2)
        ys.append(y)
    ys[peak]=noisy(10)
    #if peak<xs[-2]:
    ys[peak+2]=ys[peak+3]=noisy(5)
    lcs.append(np.stack((xs,ys)))
    classes.append("gamma")
for i in range(20):
    ys=[]
    #peak=np.random.choice(xs[:-3])
    for x in xs:
        y=noisy(2)
        ys.append(y)
    #ys[peak]=noisy(10)
    #if peak<xs[-2]:
    #ys[peak+2]=ys[peak+3]=noisy(5)
    lcs.append(np.stack((xs,ys)))
    classes.append("delta")
ids=[]
for i in range(len(classes)):
    ids.append(i)
ob_state = {}
for i, ob in enumerate(classes):
    ob_state[i] = ob

In [11]:
tested_classes=[]
best_shapelets=[]
time_res=1
for n_donor, lc_donor in enumerate(lcs):
    #Create lists with classifications of all time-series relative to the donor time series; one that the pool of shapelets is generated from
    state_donor = ob_state[ids[n_donor]]
    if state_donor not in tested_classes:
        tested_classes.append(state_donor)
    else:
        continue
    belong_class=[]
    other_class=[]
    for n_lc in range(len(lcs)):
        if ob_state[ids[n_lc]] == state_donor:
            belong_class.append(n_lc)
        else:
            other_class.append(n_lc)
    #calculate the entropy of the entire set, so it can be compared to the split set later
    prop_belong = len(belong_class)/(len(belong_class)+len(other_class))
    set_entropy = -(prop_belong)*math.log2(prop_belong)-(1-prop_belong)*math.log2(1-prop_belong)
    pool=sha.generate_shapelets(lc_donor, 1, len(lc_donor[0]))#generate shapelets from the donor time-series, 
    best_gain=0#set the initial best value of information gain to 0 (improved by any split) 
    for shapelet in pool:
        skip_shapelet=False#for entropy pruning
        #set the order of distance calculations
        #pick an other_class object first and then alternate between belong and other, when one group runs out, append the rest of the other group to the end
        order=[]
        if len(belong_class)<len(other_class):alternations=len(belong_class);larger_group=other_class
        else: alternations=len(other_class); larger_group=belong_class
        for i in range(alternations):
            order.append(other_class[i])
            order.append(belong_class[i])
        for i in range(len(larger_group)-alternations):
            order.append(larger_group[-(i+1)])
        #start distance calculations
        distances=[]
        for n_lc in order:
            lc=lcs[n_lc]
            distance=sha.distance_calculation(n_lc, lc, shapelet, time_res, belong_class)
           # print(n_donor, distance, shapelet)
            distances.append(distance)
            if len(distances)>1:
                best_split=sha.best_split_point(distances, set_entropy)
                skip_shapelet=sha.entropy_pruning(best_gain, distances, best_split, len(belong_class), len(other_class), set_entropy)
                if skip_shapelet==True:
                    break
        if skip_shapelet==False:
            gain=sha.information_gain(distances, set_entropy, best_split)
            #print(shapelet)
            #print(distances)
            if gain>best_gain:
                best_gain=gain
                best_shapelet=shapelet
    best_shapelets.append((best_shapelet, best_gain, state_donor))
best_shapelets

[(array([5.0409576, 5.0409576, 5.0409576]), 0.7219280948873623, 'alpha'),
 (array([10.0052336 ,  2.01989044,  1.99030381]), 0.7219280948873623, 'beta'),
 (array([10.0052336 ,  1.98714425,  5.02424048]), 0.7219280948873623, 'gamma'),
 (array([2.0184101 , 2.00551275, 2.00845237, 1.99348864, 1.98381966,
         1.98661739, 2.01338261, 1.99      , 2.00034905, 2.00551275,
         2.00781462, 2.01576022, 1.98381966, 1.99030381, 1.98002741,
         2.00749213, 2.00104672, 1.99482362, 2.00139513, 1.99092019,
         2.00034905, 1.98361696, 1.98002741, 1.99123258, 2.01      ,
         1.98285665, 1.98172909]), 0.9709505944546686, 'delta')]